# Hello Feature Class example: using the feature classes to calculate features

This example shows how to use the Radiomics package to directly instantiate the feature classes for feature extraction. 
Note that this is not the intended standard use. For an example on the standard use with feature extractor, see the `helloRadiomics` example.

In [26]:
from __future__ import print_function
import os
import collections
import SimpleITK as sitk
import numpy
import six
import radiomics
from radiomics import firstorder, glcm, imageoperations, shape, glrlm, glszm

## Getting the test case

Test cases can be downloaded to temporary files. This is handled by the `radiomics.getTestCase()` function, which checks if the requested test case is available and if not, downloads it. It returns a tuple with the location of the image and mask of the requested test case, or (None, None) if it fails.

Alternatively, if the data is available somewhere locally, this directory can be passed as a second argument to `radiomics.getTestCase()`. If that directory does not exist or does not contain the testcase, functionality reverts to default and tries to download the test data.

If getting the test case fails, PyRadiomics will log an error explaining the cause.

In [27]:
imageName, maskName = radiomics.getTestCase('brain1')

if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

In [28]:
image = sitk.ReadImage(imageName)
mask = sitk.ReadImage(maskName)

## Preprocess the image

#### Extraction Settings

In [29]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
# settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True

#### If enabled, resample the image

In [30]:
# Resample if necessary
interpolator = settings.get('interpolator')
resampledPixelSpacing = settings.get('resampledPixelSpacing')
if interpolator is not None and resampledPixelSpacing is not None:
  image, mask = imageoperations.resampleImage(image, mask, **settings)

## Calculate features using original image

In [31]:
# Crop the image
# bb is the bounding box, upon which the image and mask are cropped
bb, correctedMask = imageoperations.checkMask(image, mask, label=1)
if correctedMask is not None:
    mask = correctedMask
croppedImage, croppedMask = imageoperations.cropToTumorMask(image, mask, bb)

### Calculate Firstorder features

In [32]:
firstOrderFeatures = firstorder.RadiomicsFirstOrder(croppedImage, croppedMask, **settings)

# Set the features to be calculated
firstOrderFeatures.enableFeatureByName('Mean', True)
# firstOrderFeatures.enableAllFeatures()

In [33]:
# Print out the docstrings of the enabled features
print('Will calculate the following first order features: ')
for f in firstOrderFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(firstOrderFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following first order features: 
Mean

    **8. Mean**

    .. math::
      \textit{mean} = \frac{1}{N_p}\displaystyle\sum^{N_p}_{i=1}{\textbf{X}(i)}

    The average gray level intensity within the ROI.
    


In [34]:
# Calculate the features and print(out result)
print('Calculating first order features...',)
result = firstOrderFeatures.execute()
print('done')

print('Calculated first order features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating first order features...
done
Calculated first order features: 
   Mean : 825.2354363065023


### Calculate Shape Features

In [35]:
shapeFeatures = shape.RadiomicsShape(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# shapeFeatures.enableFeatureByName('Volume', True)
shapeFeatures.enableAllFeatures()

In [36]:
# Print out the docstrings of the enabled features
print('Will calculate the following shape features: ')
for f in shapeFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(shapeFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following shape features: 
Maximum2DDiameterRow

    **12. Maximum 2D diameter (Row)**

    Maximum 2D diameter (Row) is defined as the largest pairwise Euclidean distance between tumor surface mesh
    vertices in the column-slice (usually the sagittal) plane.
    
VoxelVolume

    **2. Voxel Volume**

    .. math::
      V_{voxel} = \displaystyle\sum^{N_v}_{k=1}{V_k}

    The volume of the ROI :math:`V_{voxel}` is approximated by multiplying the number of voxels in the ROI by the volume
    of a single voxel :math:`V_k`. This is a less precise approximation of the volume and is not used in subsequent
    features. This feature does not make use of the mesh and is not used in calculation of other shape features.

    .. note::
      Defined in IBSI as ``Approximate Volume``.
    
Maximum3DDiameter

    **9. Maximum 3D diameter**

    Maximum 3D diameter is defined as the largest pairwise Euclidean distance between tumor surface mesh
    vertices.

    Also known as 

In [37]:
# Calculate the features and print(out result)
print('Calculating shape features...',)
result = shapeFeatures.execute()
print('done')

print('Calculated shape features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating shape features...
done
Calculated shape features: 
   VoxelVolume : 16412.658691406243
   Maximum3DDiameter : 69.60099030590368
   MeshVolume : 16147.51180013021
   MajorAxisLength : 61.99722046980879
   Sphericity : 0.4798234536231475
   LeastAxisLength : 28.584423185376522
   Elongation : 0.5621171627174109
   SurfaceVolumeRatio : 0.3987500788652454
   Maximum2DDiameterSlice : 53.59397776919529
   Flatness : 0.46105975346582634
   SurfaceArea : 6438.821603779402
   MinorAxisLength : 34.849701666854706
   Maximum2DDiameterColumn : 49.490854979101925
   Maximum2DDiameterRow : 65.88905951721043


### Calculate GLCM Features

In [38]:
glcmFeatures = glcm.RadiomicsGLCM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glcmFeatures.enableFeatureByName('SumEntropy', True)
glcmFeatures.enableAllFeatures()

In [39]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLCM features: ')
for f in glcmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glcmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLCM features: 
JointAverage

    **2. Joint Average**

    .. math::
      \textit{joint average} = \mu_x = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_g}_{j=1}{p(i,j)i}

    Returns the mean gray level intensity of the :math:`i` distribution.

    .. warning::
      As this formula represents the average of the distribution of :math:`i`, it is independent from the
      distribution of :math:`j`. Therefore, only use this formula if the GLCM is symmetrical, where
      :math:`p_x(i) = p_y(j) \text{, where } i = j`.
    
SumAverage

    **22. Sum Average**

    .. math::

      \textit{sum average} = \displaystyle\sum^{2N_g}_{k=2}{p_{x+y}(k)k}

    Sum Average measures the relationship between occurrences of pairs
    with lower intensity values and occurrences of pairs with higher intensity
    values.

    .. warning::
      When GLCM is symmetrical, :math:`\mu_x = \mu_y`, and therefore :math:`\text{Sum Average} = \mu_x + \mu_y =
      2 \mu_x = 2 *

In [40]:
# Calculate the features and print(out result)
print('Calculating GLCM features...',)
result = glcmFeatures.execute()
print('done')

print('Calculated GLCM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Calculating GLCM features...
done
Calculated GLCM features: 
   JointAverage : 16.55380772442751
   SumAverage : 33.10761544885502
   JointEntropy : 8.799696270248813
   ClusterShade : 19.605083427286676
   MaximumProbability : 0.007352392266290182
   Idmn : 0.961402169623227
   JointEnergy : 0.002893149242988865
   Contrast : 47.492125114429776
   DifferenceEntropy : 3.74406097806642
   InverseVariance : 0.19881884197093194
   DifferenceVariance : 16.65563705027098
   Idn : 0.8726052157397169
   Idm : 0.20022255640475703
   Correlation : 0.3917522006696661
   Autocorrelation : 289.5436994017259
   SumEntropy : 5.354241321485615
   MCC : 0.49904046795855606
   SumSquares : 39.05587959224222
   ClusterProminence : 27995.937591943148
   Imc2 : 0.6942249020670357
   Imc1 : -0.09438938808738298
   DifferenceAverage : 5.284468789866316
   Id : 0.28722572382985156
   ClusterTendency : 108.73139325453903


### Calculate GLRLM Features

In [41]:
glrlmFeatures = glrlm.RadiomicsGLRLM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glrlmFeatures.enableFeatureByName('ShortRunEmphasis', True)
glrlmFeatures.enableAllFeatures()

In [42]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLRLM features: ')
for f in glrlmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glrlmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLRLM features: 
ShortRunLowGrayLevelEmphasis

    **13. Short Run Low Gray Level Emphasis (SRLGLE)**

    .. math::
      \textit{SRLGLE} = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2j^2}}}{N_r(\theta)}

    SRLGLE measures the joint distribution of shorter run lengths with lower gray-level values.
    
GrayLevelVariance

    **8. Gray Level Variance (GLV)**

    .. math::
      \textit{GLV} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)(i - \mu)^2}

    Here, :math:`\mu = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_r}_{j=1}{p(i,j|\theta)i}`

    GLV measures the variance in gray level intensity for the runs.
    
LowGrayLevelRunEmphasis

    **11. Low Gray Level Run Emphasis (LGLRE)**

    .. math::
      \textit{LGLRE} = \frac{\sum^{N_g}_{i=1}\sum^{N_r}_{j=1}{\frac{\textbf{P}(i,j|\theta)}{i^2}}}{N_r(\theta)}

    LGLRE measures the distribution of low gray-level values, with a higher value in

In [43]:
# Calculate the features and print(out result)
print('Calculating GLRLM features...',)
result = glrlmFeatures.execute()
print('done')

print('Calculated GLRLM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating GLRLM features...
done
Calculated GLRLM features: 
   ShortRunLowGrayLevelEmphasis : 0.008229766244155428
   GrayLevelVariance : 39.118151021979244
   LowGrayLevelRunEmphasis : 0.008600397891661503
   GrayLevelNonUniformityNormalized : 0.04514123814981055
   RunVariance : 0.08479457789590625
   GrayLevelNonUniformity : 175.6351923150419
   LongRunEmphasis : 1.2268440382584342
   ShortRunHighGrayLevelEmphasis : 268.9741798411307
   RunLengthNonUniformity : 3500.0432315746298
   ShortRunEmphasis : 0.9559391731405504
   LongRunHighGrayLevelEmphasis : 341.2865790983503
   RunPercentage : 0.9404064632491029
   LongRunLowGrayLevelEmphasis : 0.010601170478748765
   RunEntropy : 4.915038003159503
   HighGrayLevelRunEmphasis : 281.066493908972
   RunLengthNonUniformityNormalized : 0.8950494659480998


### Calculate GLSZM Features

In [44]:
glszmFeatures = glszm.RadiomicsGLSZM(croppedImage, croppedMask, **settings)

# Set the features to be calculated
# glszmFeatures.enableFeatureByName('LargeAreaEmphasis', True)
glszmFeatures.enableAllFeatures()

In [45]:
# Print out the docstrings of the enabled features
print('Will calculate the following GLSZM features: ')
for f in glszmFeatures.enabledFeatures.keys():
  print(f)
  print(getattr(glszmFeatures, 'get%sFeatureValue' % f).__doc__)

Will calculate the following GLSZM features: 
GrayLevelVariance

    **8. Gray Level Variance (GLV)**

    .. math::
      \textit{GLV} = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_s}_{j=1}{p(i,j)(i - \mu)^2}

    Here, :math:`\mu = \displaystyle\sum^{N_g}_{i=1}\displaystyle\sum^{N_s}_{j=1}{p(i,j)i}`

    GLV measures the variance in gray level intensities for the zones.
    
SmallAreaHighGrayLevelEmphasis

    **14. Small Area High Gray Level Emphasis (SAHGLE)**

    .. math::
      \textit{SAHGLE} = \frac{\sum^{N_g}_{i=1}\sum^{N_s}_{j=1}{\frac{\textbf{P}(i,j)i^2}{j^2}}}{N_z}

    SAHGLE measures the proportion in the image of the joint distribution of smaller size zones with higher gray-level
    values.
    
GrayLevelNonUniformityNormalized

    **4. Gray Level Non-Uniformity Normalized (GLNN)**

    .. math::
      \textit{GLNN} = \frac{\sum^{N_g}_{i=1}\left(\sum^{N_s}_{j=1}{\textbf{P}(i,j)}\right)^2}{N_z^2}

    GLNN measures the variability of gray-level intensity values i

In [46]:
# Calculate the features and print(out result)
print('Calculating GLSZM features...',)
result = glszmFeatures.execute()
print('done')

print('Calculated GLSZM features: ')
for (key, val) in six.iteritems(result):
  print('  ', key, ':', val)

Calculating GLSZM features...
done
Calculated GLSZM features: 
   GrayLevelVariance : 40.60313992393263
   ZoneVariance : 8.721239097486347
   GrayLevelNonUniformityNormalized : 0.044057307901283996
   SizeZoneNonUniformityNormalized : 0.3997843804512568
   SizeZoneNonUniformity : 747.5967914438503
   GrayLevelNonUniformity : 82.38716577540107
   LargeAreaEmphasis : 13.615508021390374
   SmallAreaHighGrayLevelEmphasis : 193.438051925864
   ZonePercentage : 0.4520183708000967
   LargeAreaLowGrayLevelEmphasis : 0.12723841553344326
   LargeAreaHighGrayLevelEmphasis : 3514.7614973262034
   HighGrayLevelZoneEmphasis : 288.6235294117647
   SmallAreaEmphasis : 0.6564478999587141
   LowGrayLevelZoneEmphasis : 0.009100942027706215
   ZoneEntropy : 6.50821498619819
   SmallAreaLowGrayLevelEmphasis : 0.006416982055097711


## Calculate Features using Laplacian of Gaussian Filter

Calculating features on filtered images is very similar to calculating features on the original image. All filters in PyRadiomics have the same input and output signature, and there is even one for applying no filter. This enables to loop over a list of requested filters and apply them in the same piece of code. It is applied like this in the execute function in feature extractor. The input for the filters is the image, with additional keywords. If no additional keywords are supplied, the filter uses default values where applicable. It returns a [generator object](https://docs.python.org/2/reference/simple_stmts.html?#yield), allowing to define the generators to be applied before the filters functions are actually called.

### Calculate Firstorder on LoG filtered images

In [47]:
logFeatures = {}
sigmaValues = [1.0, 3.0, 5.0]
for logImage, imageTypename, inputSettings in imageoperations.getLoGImage(image, mask, sigma=sigmaValues):
  logImage, croppedMask = imageoperations.cropToTumorMask(logImage, mask, bb)
  logFirstorderFeatures = firstorder.RadiomicsFirstOrder(logImage, croppedMask, **inputSettings)
  logFirstorderFeatures.enableAllFeatures()
  logFeatures[imageTypename] = logFirstorderFeatures.execute()

In [48]:
# Show result
for sigma, features in six.iteritems(logFeatures):
  for (key, val) in six.iteritems(features):
    laplacianFeatureName = '%s_%s' % (str(sigma), key)
    print('  ', laplacianFeatureName, ':', val)

   log-sigma-3-0-mm-3D_InterquartileRange : 103.15813827514648
   log-sigma-3-0-mm-3D_Skewness : -0.49838634399478493
   log-sigma-3-0-mm-3D_Uniformity : 0.09064784923477494
   log-sigma-3-0-mm-3D_Median : -73.3129653930664
   log-sigma-3-0-mm-3D_Energy : 56119386.49315185
   log-sigma-3-0-mm-3D_RobustMeanAbsoluteDeviation : 43.37792439842861
   log-sigma-3-0-mm-3D_MeanAbsoluteDeviation : 64.33120246325181
   log-sigma-3-0-mm-3D_TotalEnergy : 222641609.01213792
   log-sigma-3-0-mm-3D_Maximum : 114.29669189453125
   log-sigma-3-0-mm-3D_RootMeanSquared : 116.46989846118638
   log-sigma-3-0-mm-3D_90Percentile : 13.917341041564946
   log-sigma-3-0-mm-3D_Minimum : -354.3352355957031
   log-sigma-3-0-mm-3D_Entropy : 3.72121444058449
   log-sigma-3-0-mm-3D_Range : 468.6319274902344
   log-sigma-3-0-mm-3D_Variance : 6720.0665187057575
   log-sigma-3-0-mm-3D_10Percentile : -197.0173400878906
   log-sigma-3-0-mm-3D_Kurtosis : 3.1833658319698688
   log-sigma-3-0-mm-3D_Mean : -82.73554694841455
  

## Calculate Features using Wavelet filter

### Calculate Firstorder on filtered images

In [49]:
waveletFeatures = {}
for decompositionImage, decompositionName, inputSettings in imageoperations.getWaveletImage(image, mask):
  decompositionImage, croppedMask = imageoperations.cropToTumorMask(decompositionImage, mask, bb)
  waveletFirstOrderFeaturs = firstorder.RadiomicsFirstOrder(decompositionImage, croppedMask, **inputSettings)
  waveletFirstOrderFeaturs.enableAllFeatures()

  print('Calculate firstorder features with ', decompositionName)
  waveletFeatures[decompositionName] = waveletFirstOrderFeaturs.execute()

Calculate firstorder features with  wavelet-HLL
Calculate firstorder features with  wavelet-LHL
Calculate firstorder features with  wavelet-LHH
Calculate firstorder features with  wavelet-LLH
Calculate firstorder features with  wavelet-HLH
Calculate firstorder features with  wavelet-HHH
Calculate firstorder features with  wavelet-HHL
Calculate firstorder features with  wavelet-LLL


In [50]:
# Show result
for decompositionName, features in six.iteritems(waveletFeatures):
  for (key, val) in six.iteritems(features):
    waveletFeatureName = '%s_%s' % (str(decompositionName), key)
    print('  ', waveletFeatureName, ':', val)

   wavelet-LLL_InterquartileRange : 550.5942674265389
   wavelet-LLL_Skewness : 0.22884642646488648
   wavelet-LLL_Uniformity : 0.019907776727807427
   wavelet-LLL_Median : 2244.886736086164
   wavelet-LLL_Energy : 21547005906.071228
   wavelet-LLL_RobustMeanAbsoluteDeviation : 220.73969717214538
   wavelet-LLL_MeanAbsoluteDeviation : 293.1439959437339
   wavelet-LLL_TotalEnergy : 85483116692.78741
   wavelet-LLL_Maximum : 3180.639186772004
   wavelet-LLL_RootMeanSquared : 2282.184255507612
   wavelet-LLL_90Percentile : 2739.69052111044
   wavelet-LLL_Minimum : 1468.0772410291193
   wavelet-LLL_Entropy : 5.783004890516309
   wavelet-LLL_Range : 1712.5619457428847
   wavelet-LLL_Variance : 122620.56279600722
   wavelet-LLL_10Percentile : 1812.6847348869353
   wavelet-LLL_Kurtosis : 2.2736564306682845
   wavelet-LLL_Mean : 2255.1595095005646
   wavelet-HHH_InterquartileRange : 20.31927464217497
   wavelet-HHH_Skewness : -0.06881127372370828
   wavelet-HHH_Uniformity : 0.38291997981392567